In [1]:
import os, socket, requests
import struct
import random
import pathlib
import shapefile
from shapely.geometry import Point, Polygon
import zlib
import concurrent.futures
import traceback

# os.chdir('/root/py/m2')
# os.chdir('/root/bt/new/map')

kDefaultKey = (
    b"\x45\xf4\xbd\x0b\x79\xe2\x6a\x45\x22\x05\x92\x2c"
    b"\x17\xcd\x06\x71\xf8\x49\x10\x46\x67\x51\x00\x42"
    b"\x25\xc6\xe8\x61\x2c\x66\x29\x08\xc6\x34\xdc\x6a"
    b"\x62\x25\x79\x0a\x77\x1d\x6d\x69\xd6\xf0\x9c\x6b"
    b"\x93\xa1\xbd\x4e\x75\xe0\x41\x04\x5b\xdf\x40\x56"
    b"\x0c\xd9\xbb\x72\x9b\x81\x7c\x10\x33\x53\xee\x4f"
    b"\x6c\xd4\x71\x05\xb0\x7b\xc0\x7f\x45\x03\x56\x5a"
    b"\xad\x77\x55\x65\x0b\x33\x92\x2a\xac\x19\x6c\x35"
    b"\x14\xc5\x1d\x30\x73\xf8\x33\x3e\x6d\x46\x38\x4a"
    b"\xb4\xdd\xf0\x2e\xdd\x17\x75\x16\xda\x8c\x44\x74"
    b"\x22\x06\xfa\x61\x22\x0c\x33\x22\x53\x6f\xaf\x39"
    b"\x44\x0b\x8c\x0e\x39\xd9\x39\x13\x4c\xb9\xbf\x7f"
    b"\xab\x5c\x8c\x50\x5f\x9f\x22\x75\x78\x1f\xe9\x07"
    b"\x71\x91\x68\x3b\xc1\xc4\x9b\x7f\xf0\x3c\x56\x71"
    b"\x48\x82\x05\x27\x55\x66\x59\x4e\x65\x1d\x98\x75"
    b"\xa3\x61\x46\x7d\x61\x3f\x15\x41\x00\x9f\x14\x06"
    b"\xd7\xb4\x34\x4d\xce\x13\x87\x46\xb0\x1a\xd5\x05"
    b"\x1c\xb8\x8a\x27\x7b\x8b\xdc\x2b\xbb\x4d\x67\x30"
    b"\xc8\xd1\xf6\x5c\x8f\x50\xfa\x5b\x2f\x46\x9b\x6e"
    b"\x35\x18\x2f\x27\x43\x2e\xeb\x0a\x0c\x5e\x10\x05"
    b"\x10\xa5\x73\x1b\x65\x34\xe5\x6c\x2e\x6a\x43\x27"
    b"\x63\x14\x23\x55\xa9\x3f\x71\x7b\x67\x43\x7d\x3a"
    b"\xaf\xcd\xe2\x54\x55\x9c\xfd\x4b\xc6\xe2\x9f\x2f"
    b"\x28\xed\xcb\x5c\xc6\x2d\x66\x07\x88\xa7\x3b\x2f"
    b"\x18\x2a\x22\x4e\x0e\xb0\x6b\x2e\xdd\x0d\x95\x7d"
    b"\x7d\x47\xba\x43\xb2\x11\xb2\x2b\x3e\x4d\xaa\x3e"
    b"\x7d\xe6\xce\x49\x89\xc6\xe6\x78\x0c\x61\x31\x05"
    b"\x2d\x01\xa4\x4f\xa5\x7e\x71\x20\x88\xec\x0d\x31"
    b"\xe8\x4e\x0b\x00\x6e\x50\x68\x7d\x17\x3d\x08\x0d"
    b"\x17\x95\xa6\x6e\xa3\x68\x97\x24\x5b\x6b\xf3\x17"
    b"\x23\xf3\xb6\x73\xb3\x0d\x0b\x40\xc0\x9f\xd8\x04"
    b"\x51\x5d\xfa\x1a\x17\x22\x2e\x15\x6a\xdf\x49\x00"
    b"\xb9\xa0\x77\x55\xc6\xef\x10\x6a\xbf\x7b\x47\x4c"
    b"\x7f\x83\x17\x05\xee\xdc\xdc\x46\x85\xa9\xad\x53"
    b"\x07\x2b\x53\x34\x06\x07\xff\x14\x94\x59\x19\x02"
    b"\xe4\x38\xe8\x31\x83\x4e\xb9\x58\x46\x6b\xcb\x2d"
    b"\x23\x86\x92\x70\x00\x35\x88\x22\xcf\x31\xb2\x26"
    b"\x2f\xe7\xc3\x75\x2d\x36\x2c\x72\x74\xb0\x23\x47"
    b"\xb7\xd3\xd1\x26\x16\x85\x37\x72\xe2\x00\x8c\x44"
    b"\xcf\x10\xda\x33\x2d\x1a\xde\x60\x86\x69\x23\x69"
    b"\x2a\x7c\xcd\x4b\x51\x0d\x95\x54\x39\x77\x2e\x29"
    b"\xea\x1b\xa6\x50\xa2\x6a\x8f\x6f\x50\x99\x5c\x3e"
    b"\x54\xfb\xef\x50\x5b\x0b\x07\x45\x17\x89\x6d\x28"
    b"\x13\x77\x37\x1d\xdb\x8e\x1e\x4a\x05\x66\x4a\x6f"
    b"\x99\x20\xe5\x70\xe2\xb9\x71\x7e\x0c\x6d\x49\x04"
    b"\x2d\x7a\xfe\x72\xc7\xf2\x59\x30\x8f\xbb\x02\x5d"
    b"\x73\xe5\xc9\x20\xea\x78\xec\x20\x90\xf0\x8a\x7f"
    b"\x42\x17\x7c\x47\x19\x60\xb0\x16\xbd\x26\xb7\x71"
    b"\xb6\xc7\x9f\x0e\xd1\x33\x82\x3d\xd3\xab\xee\x63"
    b"\x99\xc8\x2b\x53\xa0\x44\x5c\x71\x01\xc6\xcc\x44"
    b"\x1f\x32\x4f\x3c\xca\xc0\x29\x3d\x52\xd3\x61\x19"
    b"\x58\xa9\x7d\x65\xb4\xdc\xcf\x0d\xf4\x3d\xf1\x08"
    b"\xa9\x42\xda\x23\x09\xd8\xbf\x5e\x50\x49\xf8\x4d"
    b"\xc0\xcb\x47\x4c\x1c\x4f\xf7\x7b\x2b\xd8\x16\x18"
    b"\xc5\x31\x92\x3b\xb5\x6f\xdc\x6c\x0d\x92\x88\x16"
    b"\xd1\x9e\xdb\x3f\xe2\xe9\xda\x5f\xd4\x84\xe2\x46"
    b"\x61\x5a\xde\x1c\x55\xcf\xa4\x00\xbe\xfd\xce\x67"
    b"\xf1\x4a\x69\x1c\x97\xe6\x20\x48\xd8\x5d\x7f\x7e"
    b"\xae\x71\x20\x0e\x4e\xae\xc0\x56\xa9\x91\x01\x3c"
    b"\x82\x1d\x0f\x72\xe7\x76\xec\x29\x49\xd6\x5d\x2d"
    b"\x83\xe3\xdb\x36\x06\xa9\x3b\x66\x13\x97\x87\x6a"
    b"\xd5\xb6\x3d\x50\x5e\x52\xb9\x4b\xc7\x73\x57\x78"
    b"\xc9\xf4\x2e\x59\x07\x95\x93\x6f\xd0\x4b\x17\x57"
    b"\x19\x3e\x27\x27\xc7\x60\xdb\x3b\xed\x9a\x0e\x53"
    b"\x44\x16\x3e\x3f\x8d\x92\x6d\x77\xa2\x0a\xeb\x3f"
    b"\x52\xa8\xc6\x55\x5e\x31\x49\x37\x85\xf4\xc5\x1f"
    b"\x26\x2d\xa9\x1c\xbf\x8b\x27\x54\xda\xc3\x6a\x20"
    b"\xe5\x2a\x78\x04\xb0\xd6\x90\x70\x72\xaa\x8b\x68"
    b"\xbd\x88\xf7\x02\x5f\x48\xb1\x7e\xc0\x58\x4c\x3f"
    b"\x66\x1a\xf9\x3e\xe1\x65\xc0\x70\xa7\xcf\x38\x69"
    b"\xaf\xf0\x56\x6c\x64\x49\x9c\x27\xad\x78\x74\x4f"
    b"\xc2\x87\xde\x56\x39\x00\xda\x77\x0b\xcb\x2d\x1b"
    b"\x89\xfb\x35\x4f\x02\xf5\x08\x51\x13\x60\xc1\x0a"
    b"\x5a\x47\x4d\x26\x1c\x33\x30\x78\xda\xc0\x9c\x46"
    b"\x47\xe2\x5b\x79\x60\x49\x6e\x37\x67\x53\x0a\x3e"
    b"\xe9\xec\x46\x39\xb2\xf1\x34\x0d\xc6\x84\x53\x75"
    b"\x6e\xe1\x0c\x59\xd9\x1e\xde\x29\x85\x10\x7b\x49"
    b"\x49\xa5\x77\x79\xbe\x49\x56\x2e\x36\xe7\x0b\x3a"
    b"\xbb\x4f\x03\x62\x7b\xd2\x4d\x31\x95\x2f\xbd\x38"
    b"\x7b\xa8\x4f\x21\xe1\xec\x46\x70\x76\x95\x7d\x29"
    b"\x22\x78\x88\x0a\x90\xdd\x9d\x5c\xda\xde\x19\x51"
    b"\xcf\xf0\xfc\x59\x52\x65\x7c\x33\x13\xdf\xf3\x48"
    b"\xda\xbb\x2a\x75\xdb\x60\xb2\x02\x15\xd4\xfc\x19"
    b"\xed\x1b\xec\x7f\x35\xa8\xff\x28\x31\x07\x2d\x12"
    b"\xc8\xdc\x88\x46\x7c\x8a\x5b\x22")

class Printable():
    cls = 'Printable'
    def __repr__(self):
        atts = {}
        for i in dir(self):
            if i[0] == '_' or i in ('main', 'sub', 'reg', 'cls'):
                continue
            att = self.__getattribute__(i)
            if inspect.ismethod(att):
                continue
            atts[i] = att
        return '%s ==> %s'%(self.cls,atts.__repr__())  

class KhDataHeader(Printable):
    cls = 'KhDataHeader'
    def __init__(self):
        pass
    def pull(self, buf):
          (self.magic_id, self.data_type_id, self.version, self.num_instances,
           self.data_instance_size, self.data_buffer_offset,self.data_buffer_size, self.meta_buffer_size
           ) = struct.unpack('<IIIiiiii', buf)

class KhQuantum(Printable):
    cls = 'KhQuantum'
    def pull(self, buf):
        (self.children, _, self.cnode_version, self.image_version, self.terrain_version, self.num_channels, _,
        self.type_offsset, self.version_offset, self.image_neiphors, self.image_provider, self.terrain_provider, _
        ) = struct.unpack('<BBHHHHHiiQBBH', buf)
        self._subnodes = [None, None, None, None]
        self._parrent = None
    def next_slot(self):
        for i in range(4):
            if 1<<i & self.children and (not self._subnodes[i]):
                return i
        return -1
    def next_node(self):
        ret = self
        while True:
            if ret.next_slot() >= 0:
                return ret
            ret = ret._parrent

def decode_qtpacket(buf):
    header = KhDataHeader()
    header.pull(buf[:32])
    items = []
    p = 32
    current_node = None
    root_node = None
    for i in range(header.num_instances):
        q = KhQuantum()
        q.pull(buf[p:p+32])
        p += 32
        if i == 0:
            root_node = q
        else:
            cursor = current_node.next_node()
            q._parrent = cursor
            cursor._subnodes[cursor.next_slot()] = q
        current_node = q
    return header, root_node

def encrypt(raw, key = kDefaultKey):
    if len(raw) == 0 or len(key) == 0:
        return raw
    ret = bytearray(raw)
    dp = 0
    dpend = len(raw)
    op = 0
    kpstart = 0
    kpend = len(key)
    kp = 0
    off = 8
    end_2 = dpend & ~7
    
    while dp < end_2:
        off = (off + 8) % 24
        kp = kpstart + off
        
        while (dp < end_2) and (kp < kpend):
            for i in range(8):
                ret[op+i] = raw[dp+i] ^ key[kp+i]
            dp += 8
            op += 8
            kp += 24
            
    if dp < dpend:
        if kp >= kpend:
            off = (off + 8) % 24
            kp = kpstart + off
        while dp < dpend:
            ret[op] = raw[dp] ^ key[kp]
            dp += 1
            op += 1
            kp += 1
    return bytes(ret)

def decrypt(raw, key = kDefaultKey):
    return encrypt(raw, key)

class HttpClient():
    def __init__(self):
        self.authed = False
        self.sk = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
        self.addr = ('localhost', 20009)
        self.s = requests.session()
        self.Headers = {
                        'User-Agent':       'GoogleEarth/7.3.2.5776(Windows;Microsoft Windows (6.2.9200.0);en;kml:2.2;client:Pro;type:default)',
                        'Connection':       'Keep-Alive',
                        'Accept-Encoding':  'gzip, deflate',
                        'Accept-Language':  'zh-CN,en,*',
                        'Accept': None
                        }
        self.Auth_Header = {
                            'Accept': 'application/vnd.google-earth.kml+xml, application/vnd.google-earth.kmz, image/*, */*',
                            'Content-Type': 'application/octet-stream',
                            'User-Agent':'GoogleEarth/7.3.2.5776(Windows;Microsoft Windows (6.2.9200.0);en;kml:2.2;client:Pro;type:default)',
                            'Content-Length': '49',
                            'Connection': 'Keep-Alive',
                            'Accept-Encoding': 'gzip, deflate',
                            'Accept-Language': 'zh-CN,en,*'
                        }

    def auth(self):
        buf = struct.pack('HHHH', random.randint(0,255*255),random.randint(0,255*255),
                          random.randint(0,255*255),random.randint(0,255*255))
        self.sk.sendto(buf, self.addr)
        auth_data,_ = self.sk.recvfrom(1024)
        resp = requests.request('POST', 'https://kh.google.com/geauth?ct=pro', data=auth_data,headers=self.Auth_Header)
        if resp.status_code != 200:
            print(resp.status_code, resp.content)
        assert resp.status_code == 200
        raw = resp.content
        cookie_len, = struct.unpack('>i',raw[4:8])
        self.cookie = raw[8:8+cookie_len].decode('utf-8')
        self.authed = True
        self.Headers['Cookie'] = '$Version="0"; SessionId=%s; State=1'%self.cookie
    def get(self, url):
        if not self.authed:
            self.auth()
#         resp = requests.request('GET', url, headers=Headers)
        resp = self.s.request('GET', url, headers=self.Headers)
        if resp.status_code == 200:
            return 200, resp.content
        if resp.status_code == 403:
            self.authed = False
            self.auth()
            resp = self.s.request('GET', url, headers=self.Headers)
            if resp.status_code == 200:
                return 200, resp.content
#             print(resp.status_code, resp.content)
            return resp.status_code, None
#         print(resp.status_code, url)
        return resp.status_code, None

class Co_Range():
    def __init__(self, x1, x2, y1, y2):
        self.x1, self.x2, self.y1, self.y2 = x1, x2, y1, y2
    def __repr__(self):
        return ((self.x1, self.x2) , (self.y1, self.y2)).__repr__()
    def contains(self, point):
        x, y = point
        return x >= self.x1 and x < self.x2 and y >= self.y1 and y < self.y2

def blist_2_co_range(blist):
    ret = Co_Range(-180,180,-180,180)
    for i in blist[1:]: # first letter is always 0
        if i == '0':
            ret = Co_Range(ret.x1, ret.x1 + (ret.x2-ret.x1)/2,
                           ret.y1, ret.y1 + (ret.y2-ret.y1)/2)
        if i == '1':
            ret = Co_Range(ret.x1 + (ret.x2-ret.x1)/2, ret.x2,
                           ret.y1, ret.y1 + (ret.y2-ret.y1)/2)
        if i == '2':
            ret = Co_Range(ret.x1 + (ret.x2-ret.x1)/2, ret.x2,
                           ret.y1 + (ret.y2-ret.y1)/2, ret.y2)
        if i == '3':
            ret = Co_Range(ret.x1, ret.x1 + (ret.x2-ret.x1)/2,
                           ret.y1 + (ret.y2-ret.y1)/2, ret.y2)
    return ret

def point_2_blist(point, level):
    x, y = point
    ret = Co_Range(-180,180,-180,180)
    r = '0'
    for i in range(level - 1):
        ret0 = Co_Range(ret.x1, ret.x1 + (ret.x2-ret.x1)/2,
                        ret.y1, ret.y1 + (ret.y2-ret.y1)/2)
        if ret0.contains(point):
            r += '0'
            ret = ret0
            continue
        ret1 = Co_Range(ret.x1 + (ret.x2-ret.x1)/2, ret.x2,
                       ret.y1, ret.y1 + (ret.y2-ret.y1)/2)
        if ret1.contains(point):
            r += '1'
            ret = ret1
            continue
        ret2 = Co_Range(ret.x1 + (ret.x2-ret.x1)/2, ret.x2,
                       ret.y1 + (ret.y2-ret.y1)/2, ret.y2)
        if ret2.contains(point):
            r += '2'
            ret = ret2
            continue
        ret3 = Co_Range(ret.x1, ret.x1 + (ret.x2-ret.x1)/2,
                        ret.y1 + (ret.y2-ret.y1)/2, ret.y2)
        if ret3.contains(point):
            r += '3'
            ret = ret3
            continue
        assert(False)
    return r

class Blist():
    def __init__(self, blist):
        self.blist = blist
        self.co = blist_2_co_range(blist)
        co = self.co
        self.border_len = co.x2 - co.x1
        self.center = (co.x1+co.x2)/2, (co.y1+co.y2)/2
    def right(self):
        return point_2_blist((self.center[0]+self.border_len, self.center[1]), len(self.blist))
    def left(self):
        return point_2_blist((self.center[0]-self.border_len, self.center[1]), len(self.blist))
    def up(self):
        return point_2_blist((self.center[0], self.center[1]+self.border_len), len(self.blist))
    def down(self):
        return point_2_blist((self.center[0], self.center[1]-self.border_len), len(self.blist))

def blist_2_path(blist):
    ret = []
    while blist:
        if len(blist) < 4:
            break
        ret.append(blist[:4])
        blist = blist[4:]
    return '/'.join(ret)

def intersect(blist, poly):
    co_range = blist_2_co_range(blist)
    if Point(co_range.x1, co_range.y1).within(poly):
        return True
    if Point(co_range.x1, co_range.y2).within(poly):
        return True
    if Point(co_range.x2, co_range.y1).within(poly):
        return True
    if Point(co_range.x2, co_range.y2).within(poly):
        return True
    return False

def inorder_download(httpclient, node, path_name, prefix, tree_path):
    flags = node.children
    if flags & 0x40: ## have image
        fn = '%s%s.%d.JFIF'%(prefix, tree_path, node.image_version)
        fn = os.path.join(path_name, fn)
        if not os.path.exists(fn):
#             print('Download %s'%fn)
            url = 'https://kh.google.com/flatfile?f1-%s%s-i.%s'%(prefix, tree_path, node.image_version)
            code, img = httpclient.get(url)
            img = decrypt(img)
            open(fn, 'wb').write(img)
    if flags & 0x10: ## have subtree
        assert len(tree_path) == 4
#         print('####################################')
        download_q2_2(httpclient, prefix+tree_path, node.cnode_version)
    for i in range(4):
        subnode = node._subnodes[i]
        if subnode:
            inorder_download(httpclient, subnode, path_name, prefix, '%s%d'%(tree_path,i))

def download_q2_2(httpclient, levels, ver=876):
    p = blist_2_path(levels)
    if not os.path.exists(p):
        pathlib.Path(p).mkdir(parents=True, exist_ok=True)
    fn = '%s-q.%d' % (levels, ver)
    fn = os.path.join(p, fn)
    if os.path.exists(fn+'.404'):
        return
    if not os.path.exists(fn):
        code, qp = httpclient.get('https://kh.google.com/flatfile?q2-%s-q.%d'%(levels,ver))
        if code == 404:
            open(fn+'.404', 'wb')
            return
        qp = decrypt(qp)
        open(fn, 'wb').write(qp)
    else:
        qp = open(fn, 'rb').read()
    qp = zlib.decompress(qp[8:])
    header, qtp = decode_qtpacket(qp)
    inorder_download(httpclient, qtp, p, levels, '')
    
def download_q2(levels, ver=876):
    try:
#         print('hello')
        hc = HttpClient()
        download_q2_2(hc, levels, ver)
        print('%s Done'%levels)
    except:
        traceback.print_exc()

def download_range(shape_file):
    sf = shapefile.Reader(shape_file)
    shape = sf.shapes()[0]
    poly = Polygon(shape.points)
#     print(shape.bbox)
    bbox = shape.bbox
    left_bottom = bbox[0], bbox[1]
    left_bottom_blist = point_2_blist(left_bottom, 16)
    co_r = blist_2_co_range(left_bottom_blist)
    step = co_r.x2 - co_r.x1
    x_n = int((bbox[2] - bbox[0])/step) + 1
    y_n = int((bbox[3] - bbox[1])/step) + 1
    left_blist = left_bottom_blist
    download_works = []
    for i in range(x_n):
        p = left_blist
        for j in range(y_n):
#             print(p)
            if intersect(p, poly):
#                 print('!!!!!!!')
#                 download_q2(p)
                download_works.append(p)
            p = Blist(p).right()
        left_blist = Blist(left_blist).up()
    print(download_works)
#     with concurrent.futures.ThreadPoolExecutor(max_workers=20) as executor:
    with concurrent.futures.ProcessPoolExecutor(max_workers=4) as executor:
        for i in download_works:
            executor.submit(download_q2, i)
    
# download_range('pudong.shp')

In [6]:
download_q2('0210202200111222')

0210202200111222 Done


In [5]:
3.814697265625e-06*16, -0.6798095703125*180

(6.103515625e-05, -122.36572265625)

In [8]:
blist_2_co_range('021310112222')

((123.57421875, 123.75), (47.63671875, 47.8125))

In [13]:
len(kDefaultKey)

1016

In [3]:
122.36572265625- 122.34375

0.02197265625

In [7]:
def calc_range(shape_file):
    sf = shapefile.Reader(shape_file)
    shape = sf.shapes()[0]
    poly = Polygon(shape.points)
#     print(shape.bbox)
    bbox = shape.bbox
    left_bottom = bbox[0], bbox[1]
    left_bottom_blist = point_2_blist(left_bottom, 12)
    co_r = blist_2_co_range(left_bottom_blist)
    step = co_r.x2 - co_r.x1
    x_n = int((bbox[2] - bbox[0])/step) + 1
    y_n = int((bbox[3] - bbox[1])/step) + 1
    left_blist = left_bottom_blist
    download_works = []
    for i in range(x_n):
        p = left_blist
        for j in range(y_n):
#             print(p)
            if intersect(p, poly):
#                 print('!!!!!!!')
#                 download_q2(p)
                download_works.append(p)
            p = Blist(p).right()
        left_blist = Blist(left_blist).up()
    return download_works

In [9]:
works = _

In [12]:
len(works)

37630

In [40]:
# es_host = 'localhost:9200'
es_host = '4.4.4.4:9200'
import elasticsearch, elasticsearch.helpers
es = elasticsearch.Elasticsearch([es_host])

req = []
n = 0
for i in levels12:
    n += 1
    if n % 1000 == 0:
        print('Got %d rows' % n)
    a = {'levels':i, 'level':len(i), 'status':'waiting', 'worker':'none'}
    b = {'_index':'map_work', '_type':'work', '_id':i, '_source':a}
    req.append(b)
    if len(req) == 1000:
        elasticsearch.helpers.bulk(client=es, actions=req)
        req = []
if req:
    elasticsearch.helpers.bulk(client=es, actions=req)
print('Done')   

Got 1000 rows
Got 2000 rows
Got 3000 rows
Got 4000 rows
Got 5000 rows
Got 6000 rows
Got 7000 rows
Got 8000 rows
Got 9000 rows
Got 10000 rows
Got 11000 rows
Got 12000 rows
Got 13000 rows
Got 14000 rows
Got 15000 rows
Got 16000 rows
Got 17000 rows
Got 18000 rows
Got 19000 rows
Got 20000 rows
Got 21000 rows
Got 22000 rows
Got 23000 rows
Got 24000 rows
Got 25000 rows
Got 26000 rows
Got 27000 rows
Got 28000 rows
Got 29000 rows
Got 30000 rows
Got 31000 rows
Got 32000 rows
Got 33000 rows
Got 34000 rows
Got 35000 rows
Got 36000 rows
Got 37000 rows
Done


In [7]:
levels12 = calc_range('china.shp')

NameError: name 'calc_range' is not defined

In [6]:
levels12

NameError: name 'levels12' is not defined

In [15]:
import gedecode
c = open('/tmp/test.img', 'rb').read()

In [16]:
decrypt(c)

b'\xc4hT\t$\x05Y\x122=\x05\x1d\xbb\x9c\xa2W\xf3\xf5\x11|\x13?\x8f!l\x0e\xf7D\x92%\x01P\xa9vUu\xb2\xed6\x07\\\xad\x04f8\xdf\x92G\xff\xf8\xf6\x1a\x84]v\x1f\x00Rp|v\xf6qq\x0b\xe4\xae_\x9b.\x08^7K\x85~1|yk\xc0K\x18\x1a\x0bn\x0eY\xa7A\x03:\xb9\x8b\x1bB\xdb\x7f\xdbFK \x87M\xc6\x1f\x1dE\xb5\xdd!\x11\xd4\x01\xf3P3V\x97raKMpG\xa8 d\x9a\xb4\xfex\xb9\xdd\xc4\'\xef+\x87\x1d4\x19\xa4_\r\x02\x1cAT\x98e(_x\xb7\x04\xe2\x0cM@\x83K\xbeS\x7f\xed5J\xb7\xeb vv\x123\x03\xa2\xccsK\xe2\xdap:p\x0e\x8a&\x86V\xd6\x15\xc4-2\x16;\xf6\xb1,\xc0\xb1\xa3!\x97R\x95mg)\x98\x15_\xe2\'-\x92\x91\xf5m\xe6\xf4\x9af\xf6\x94I)\xb4q\x0c\x1f\x89\x05\xc1Z,\xa44K#:\xcd&\xab\x085\x19\xa4\x0b\xbcI\x82\xef`7d\x12/Q\xe3\xcf\xa6p\xdc\xb9\xe2JQ\x98\xb9Z\xda\x15\xa1\x14\xc3\xa6V\x0e\x031\xef\x07s\xb6OtxJK\x15\xbb\xb2\xfb/\xdc\xcb\x03(\xe6\xe5&\x01\xd1if\x15[\x86s\x17\x96\x85"\x04\x1e\xafN\x10b\x15\x04\x08~\xb6\x9de]\xcd\x96OYs\x17#}\xc4\xcdsB\x92\x0b!A\xaf\x86\x06\x16U\x19x\xeb\x81\x8dnD\xd51|\x11\xfeQ;\x90\x19\xaf\x1bm\xbe\x7fQ\x10\x93T

In [18]:
cc = bytearray(c)

gedecode.gedecode(cc)

In [22]:
open('/tmp/abc.c', 'wb').write(cc)

2166

In [46]:
set(list(range(5)))

{0, 1, 2, 3, 4}

In [43]:
b = _

In [49]:
b.remove(3)

In [50]:
ccc = {1:2, 3:4}

In [52]:
del ccc[1]

In [62]:
ddd = [1,2,3]

ddd.pop()

3

In [65]:
ddd.pop()

IndexError: pop from empty list